# Introduction
This project aims to compare the Neighborhoods of two cities that are Startup Capitals of their respective countries. For which we have chosen, Bangalore in India and, The Silicon Valley, San Francisco in United States of America. 
The primary objective of this project is to help Startup offices in Bangalore on deciding a location in San Francisco to open a branch office. A lot of startup look to open office in The Silicone Valley, and we wish to provide them guidance via this project, by helping them find places similar to their current head office, in Bangalore.  


# Data Sources
1.	The neighborhood data of the two cities is taken from Opencity and Wikipedia, respectively. This for which a are being shared bellow :
        1. https://opencity.in/data/bbmp-wards
        1. https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco
    The data from open city is in a CSV formal while the data from Wikipedia will be required to be Scrapped before being used. Beautiful Soup library will be used to scrap the data from the Wikipedia page. 
    Bangalore has 199 neighbourhoods, and San Francisco has 114 neighbourhoods.
1.	Geopy library to be used for geocodes. 
1.	Folium library will be utilized to represent the data on maps. 
1.	Scikit-learn will be used to utilize machine learning. 
1.	Foursquare API will be used to gather neighborhood data.


# Importing Data
Importing data is divided into 3 stages. 
   1. The first stage is getting list of neighbourhoods of the four cities from the above Wikipedia links. 
   1. The second stage is getting location of neighbourhoods. 
   1. The third stage is getting the venues in the neighbourhoods from Foursquare.

## Neighbourhood lists of Cities
We already have the links of the Wikipedia pages from which we can get the list of neighbourhoods in each city. Beautiful Soup library is used to extract the information from the wikitables in the pages. This data is stored in a pandas dataframe. Along with the neighbourhoods, the city name, the state name and the country name are stored.

## Geolocation of the Neighbourhoods
The geopy library is used to get the location data of the Neighbourhoods. Now in geopy library, Nominatim service is used. 
For leveraging the free service of Nominatim there is a restriction of 1 call per sec. If the calls are within a second, then a ‘timeout’ error is displayed. Therefore, there needs to be an one sec gap between each call. A sufficient gap of 2 seconds is provided by calling the sleep function.

### Timeout Errors
Even after providing a 2 sec gap in calls, there are timeout errors. So, to handle these errors is simple. Simply call the Nominatim service again for these locations after checking for network connectivity.

### Missing/No Coordinates
Due to spelling errors in the name of some locations, it will not resolve into coordinates. These can be rectified by using different spellings. Some locations will not resolve despite that. Then that data is procured manually searching on Google Maps.


In [1]:
import pandas as pd
import numpy as np
import requests
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from bs4 import BeautifulSoup

Steps to follow : 

1. Download the Bangalore Neighbourhood data from the Link
1. Then upload the data into the BD2 SQL database in IBM could
1. After the Database has been updated then make a connection with this notebook to the Database
1. Now you can read the data into your SQL file 

_Voila!_

In [2]:
%load_ext sql

In [3]:
%sql ibm_db_sa://mdm67023:gmzv%40m0fzls7b1g9@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB

In [5]:
%%sql
select TABNAME,TABSCHEMA from SYSCAT.TABLES where TABSCHEMA='MDM67023'

 * ibm_db_sa://mdm67023:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


tabname,tabschema
PETSALE,MDM67023
INTERNATIONAL_STUDENT_TEST_SCORES,MDM67023
EMPLOYEES,MDM67023
JOB_HISTORY,MDM67023
JOBS,MDM67023
DEPARTMENTS,MDM67023
LOCATIONS,MDM67023
INSTRUCTOR,MDM67023
CHICAGO_SOCIOECONOMIC_DATA,MDM67023
CHICAGO_CRIME_DATA,MDM67023


<p>Here I go practicing HTML here </p>
<p>   These new coders are too enthusiastic :P </p>
<p>
<ul>
<li>We want to use 'BANGALORE_NEIGHBOURHOOD'</li>
<li>I had forgotten the name :P</li>
</ul>
</p>

In [12]:
BN_sql= %sql select * from BANGALORE_NEIGHBOURHOOD
df_bang = BN_sql.DataFrame()
df_bang.head()

 * ibm_db_sa://mdm67023:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


,Ward No,Ward Name,BBMP Zone Name,BBMP Division,BBMP Sub Division,Assembly constituency,MP Constituency
0,1,Kempegowda Ward,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
1,2,Chowdeswari Ward,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
2,3,Atturu,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
3,4,Yelahanka Satellite Town,Yelahanka,Yelahanka,Yelahanka,Yelahanka,Chikballapur
4,5,Jakkuru,Yelahanka,Byatarayanapura,Byatarayanapura,Byatarayanapura,Bangalore North


In [13]:
webPage = requests.get("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco").text
soup = BeautifulSoup(webPage)

In [15]:
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of neighborhoods in San Francisco - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XsG0FgpAMM4AAKN34IIAAAAG","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighborhoods_in_San_Francisco","wgTitle":"List of neighborhoods in San Francisco","wgCurRevisionId":952030286,"wgRevisionId":952030286,"wgArticleId":1487556,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: archived copy as title","Webarchive template archiveis links","Webarchive template wayback link

In [25]:
toc_tags = soup.find_all(class_='toctext')
toc_tags = toc_tags[:-4]
toc_tags [:5]

[<span class="toctext">Alamo Square</span>,
 <span class="toctext">Anza Vista</span>,
 <span class="toctext">Ashbury Heights</span>,
 <span class="toctext">Balboa Park</span>,
 <span class="toctext">Balboa Terrace</span>]

In [24]:
headlines =soup.find_all(class_='mw-headline')
headlines = headline[:-4]

In [26]:
sanfran_neighbourhood=[]
for toc_tag in headlines:
    sanfran_neighbourhood.append(hedline.text)
sanfran_neighbourhood[0:5]

['Alamo Square',
 'Anza Vista',
 'Ashbury Heights',
 'Balboa Park',
 'Balboa Terrace']